<a href="https://colab.research.google.com/github/asakhala921/Propoganda-Detection-in-News-Articles/blob/master/notebooks/technique_classification_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
!unzip -q ptc_corpus.zip

Saving ptc_corpus.zip to ptc_corpus.zip
User uploaded file "ptc_corpus.zip" with length 4570646 bytes


In [ ]:
#@title Install Dependencies
%%bash
git clone -b add_articleid_to_semeval2020_11 https://github.com/hemildesai/datasets.git
git clone https://github.com/asakhala921/Propoganda-Detection-in-News-Articles.git
cd Propoganda-Detection-in-News-Articles && pip install -qU -r requirements.txt
pip install -e .

Obtaining file:///content/Propoganda-Detection-in-News-Articles
  Running setup.py develop for ucla263brah


Cloning into 'datasets'...
Cloning into 'Propoganda-Detection-in-News-Articles'...
ERROR: pandas-profiling 2.11.0 has requirement requests>=2.24.0, but you'll have requests 2.23.0 which is incompatible.
ERROR: pandas-profiling 2.11.0 has requirement tqdm>=4.48.2, but you'll have tqdm 4.41.1 which is incompatible.
ERROR: phik 0.11.2 has requirement scipy>=1.5.2, but you'll have scipy 1.4.1 which is incompatible.


In [1]:
#@title Import Dependencies

%load_ext autoreload
%autoreload 2

import re
import random
import string
from collections import defaultdict
from functools import partial

from IPython.core.display import display, HTML

import spacy
from spacy import displacy

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

import torch
from torch.utils import data as D
import torch.nn.functional as F

SEED = 100
torch.manual_seed(SEED)

import ucla263brah.tc as tc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## EDA

In [2]:
dataset = load_dataset("./datasets/datasets/sem_eval_2020_task_11/", data_dir="ptc_corpus/datasets/")
labels, colors = tc.utils.get_labels_and_colors(dataset)

tokenizer = tc.dataset.get_tokenizer("bert-base-uncased")

Using custom data configuration default-38c395c26995e028
Reusing dataset sem_eval_2020_task_11 (/root/.cache/huggingface/datasets/sem_eval_2020_task_11/default-38c395c26995e028/1.1.0/39cf0fe55e04b30029866806211d306797c07efd265a5c4fa33cfe512adf142e)


In [ ]:
df, ldf, mismatch = tc.utils.prepare_df(dataset, tokenizer, labels)
profile1 = tc.utils.generate_report(df)
profile2 = tc.utils.generate_report(ldf)

profile1

Token indices sequence length is longer than the specified maximum sequence length for this model (667 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
html = tc.utils.visualize_example(1, dataset, labels, colors)
display(HTML(html))

## Prepare Dataset

In [3]:
train_data, dev_data, test_data = tc.dataset.prepare_data(dataset, max_length=256, prepare_fn=tc.dataset.get_spans_in_context)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=random.seed(SEED), stratify=list(map(lambda x: x["technique"], train_data)))

## Train and Predict

In [ ]:
BATCH_SIZE = 8

collate_fn = partial(tc.trainer.collate_fn_with_tokenizer, tokenizer=tokenizer)
train_dataset = tc.dataset.PTCTechniqueClassificationDataset(train_data, tokenizer)
val_dataset = tc.dataset.PTCTechniqueClassificationDataset(val_data, tokenizer)
dev_dataset = tc.dataset.PTCTechniqueClassificationDataset(dev_data, tokenizer)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    evaluation_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,  # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=1e-4,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate=5e-5,
    label_smoothing_factor=0.10
)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(labels))
trainer = tc.trainer.PTCTechniqueClassificationTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    collate_fn=collate_fn
    # compute_metrics=compute_metrics
)
trainer.train()
preds = predict_and_print_results(trainer, dev_dataset)

In [ ]:
tokenizer = get_tokenizer("roberta-large")
train_dataset = PTCTechniqueClassificationDataset(train_data, tokenizer)
val_dataset = PTCTechniqueClassificationDataset(val_data, tokenizer)
dev_dataset = PTCTechniqueClassificationDataset(dev_data, tokenizer)
collate_fn = partial(collate_fn_with_tokenizer, tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=1e-2,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate=5e-5,
    fp16=True
    # label_smoothing_factor=0.10
)

model = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels=len(labels))
trainer = PTCTechniqueClassificationTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    collate_fn=collate_fn
    # compute_metrics=compute_metrics
)
trainer.train()
preds = predict_and_print_results(trainer, dev_dataset)

In [5]:
tokenizer = tc.dataset.get_tokenizer("microsoft/deberta-large")
train_dataset = tc.dataset.PTCTechniqueClassificationDataset(train_data, tokenizer)
val_dataset = tc.dataset.PTCTechniqueClassificationDataset(val_data, tokenizer)
dev_dataset = tc.dataset.PTCTechniqueClassificationDataset(dev_data, tokenizer)
collate_fn = partial(tc.trainer.collate_fn_with_tokenizer, tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    evaluation_strategy="epoch",
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=2,   # batch size for evaluation
    gradient_accumulation_steps=4,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=1e-2,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate=5e-5,
    label_smoothing_factor=0.10,
    fp16=True,
    save_total_limit=1,
)

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-large", 
    num_labels=len(labels),
    label2id = {b: a for a, b in enumerate(labels)},
    id2label = {a: b for a, b in enumerate(labels)},
)

trainer = tc.trainer.PTCTechniqueClassificationTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    collate_fn=collate_fn
    # compute_metrics=compute_metrics
)
trainer.train()
preds = tc.predictor.predict_and_print_results(trainer, dev_dataset, labels)

Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['pooler.dense.weig

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
0,1.342200,1.553379,97.355200,6.297000
1,0.932000,1.388748,97.817100,6.267000


Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
0,1.342200,1.553379,97.355200,6.297000
1,0.932000,1.388748,97.817100,6.267000
2,0.676500,1.327259,97.546600,6.284000


                                    precision    recall  f1-score   support

               Appeal_to_Authority     0.2857    0.2857    0.2857        14
          Appeal_to_fear-prejudice     0.3265    0.3636    0.3441        44
    Bandwagon,Reductio_ad_hitlerum     1.0000    0.6000    0.7500         5
           Black-and-White_Fallacy     0.4286    0.1364    0.2069        22
         Causal_Oversimplification     0.4706    0.4444    0.4571        18
                             Doubt     0.5556    0.6061    0.5797        66
         Exaggeration,Minimisation     0.4898    0.7059    0.5783        68
                       Flag-Waving     0.8194    0.6782    0.7421        87
                   Loaded_Language     0.7629    0.8215    0.7911       325
             Name_Calling,Labeling     0.6933    0.8525    0.7647       183
                        Repetition     0.6300    0.4345    0.5143       145
                           Slogans     0.6897    0.5000    0.5797        40
       Thou

In [33]:
pred_df = tc.predictor.postprocess_predictions(dev_data, preds, labels)

                                    precision    recall  f1-score   support

               Appeal_to_Authority     0.3077    0.2857    0.2963        14
          Appeal_to_fear-prejudice     0.3265    0.3636    0.3441        44
    Bandwagon,Reductio_ad_hitlerum     1.0000    0.6000    0.7500         5
           Black-and-White_Fallacy     0.4286    0.1364    0.2069        22
         Causal_Oversimplification     0.4706    0.4444    0.4571        18
                             Doubt     0.6061    0.6061    0.6061        66
         Exaggeration,Minimisation     0.4898    0.7059    0.5783        68
                       Flag-Waving     0.8358    0.6437    0.7273        87
                   Loaded_Language     0.7934    0.8154    0.8042       325
             Name_Calling,Labeling     0.7327    0.8087    0.7688       183
                        Repetition     0.6490    0.6759    0.6622       145
                           Slogans     0.6897    0.5000    0.5797        40
       Thou

In [ ]:
tokenizer = tc.dataset.get_tokenizer("microsoft/deberta-large")
final_dataset = tc.dataset.PTCTechniqueClassificationDataset(train_data + val_data + dev_data, tokenizer)
val_dataset = tc.dataset.PTCTechniqueClassificationDataset(val_data, tokenizer)
collate_fn = partial(tc.trainer.collate_fn_with_tokenizer, tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    evaluation_strategy="epoch",
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=2,   # batch size for evaluation
    gradient_accumulation_steps=4,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=1e-2,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate=5e-5,
    label_smoothing_factor=0.10,
    fp16=True,
    save_total_limit=1,
)

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-large", 
    num_labels=len(labels),
    label2id = {b: a for a, b in enumerate(labels)},
    id2label = {a: b for a, b in enumerate(labels)},
)

trainer = tc.trainer.PTCTechniqueClassificationTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=final_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    collate_fn=collate_fn
    # compute_metrics=compute_metrics
)
trainer.train()

test_dataset = tc.dataset.PTCTechniqueClassificationDataset(test_data, tokenizer)
preds_test = tc.predictor.predict_and_print_results(trainer, test_dataset, labels)
test_df = tc.predictor.postprocess_predictions(test_data, preds_test, labels)

Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['pooler.dense.weig

Epoch,Training Loss,Validation Loss


In [ ]:
lines = [f"{row.article_ids[7:]}\t{labels[row.predictions]}\t{row.start}\t{row.end}\n" for _, row in test_df.iterrows()]

with open("./test_set_tc_output.out", "w") as f:
     f.writelines(lines)

## Utils

In [28]:
#@title Setup Colab SSH

!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password="YOLO123")

In [18]:
#@title Upload logs to tensorboard.dev

!tensorboard dev upload --logdir ./logs \
  --name "Technique Classification with DeBerta Large" \
  --description "Training results from https://colab.research.google.com/drive/1eee0W3E4A1oRgeJypa2qVUGF1pO1hjHq#scrollTo=HF78Cm333E1h" \
  --one_shot

2021-03-09 12:43:32.187410: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./logs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&sc

Tensorboard - https://tensorboard.dev/experiment/bTpSvTLNSayqBWGWUZ9NaQ/

In [50]:
#@title Save model to Huggingface Model Hub
!sudo apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/hd10/semeval2020_task11_tc

model.save_pretrained("./semeval2020_task11_tc")
tokenizer.save_pretrained("./semeval2020_task11_tc")